In [1]:
import numpy as np
import numpy.linalg as lin
import scipy.stats as sts
import scipy.optimize as opt

In [2]:
data = np.loadtxt('/Users/rubyzhang/Desktop/UChicago/OSML/BootCamp2017/Econ/Wk4_StrEst/data/MacroSeries.txt',delimiter=',')
ct = data[:,0]
kt = data[:,1]
wt = data[:,2]
rt = data[:,3]

In [17]:
def data_moments(wt,kt,ct,rt,alpha,beta,rho,mu):
    zt = np.log(rt/(alpha*(kt**(alpha-1))))
    data_moms = np.empty(4)
    
    et = np.zeros(len(zt)-1)
    et = zt[1:]-rho*zt[:-1]-(1-rho)*mu
    data_moms[0] = np.mean(et)
    
    et2 = np.zeros(len(zt)-1)
    et2 = et*zt[:-1]
    data_moms[1] = np.mean(et2)
    
    mc1 = np.zeros(len(zt)-1)
    mc1 = (beta*alpha*np.exp(zt[1:])*(kt[1:]**(alpha-1))*ct[:-1]/ct[1:])-1
    data_moms[2] = np.mean(mc1)
    
    mc2 = np.zeros(len(zt)-1)
    mc2 = mc1*wt[:-1]
    data_moms[3] = np.mean(mc2)
    
    return data_moms

def err_vec(wt,kt,ct,rt,alpha,beta,rho,mu,simple):
    moms_data = data_moments(wt,kt,ct,rt,alpha,beta,rho,mu)
    moms_model = np.zeros_like(moms_data)
    if simple:
        err_vec = moms_model - moms_data
    else:
        err_vec = (moms_model - moms_data) / moms_data
    return err_vec

def criterion(params, *args):
    alpha,beta,rho,mu = params
    wt,kt,ct,rt = args
    W = np.eye(4)
    err = err_vec(wt,kt,ct,rt,alpha,beta,rho,mu,simple=True)
    crit_val = np.dot(np.dot(err.T, W), err) 
    return crit_val 

alpha_init = 0.25
beta_init = 0.60
rho_init = -0.000004
mu_init = 1
params_init = np.array([alpha_init, beta_init, rho_init, mu_init])
args = (wt,kt,ct,rt)
bounds = ((1e-10,1-1e-10),(1e-10,1-1e-10),(-1+1e-10,1-1e-10),(1e-10,None))
results = opt.minimize(criterion, params_init, args=args, bounds=bounds)
alpha_MLE, beta_MLE, rho_MLE, mu_MLE= results.x
print(results)
print('alpha_MLE=', alpha_MLE, 'beta_MLE', beta_MLE, 'rho_MLE=', rho_MLE, 'mu_MLE=', mu_MLE)
print('Minimized Criterion:', criterion(results.x, *args))

      fun: 0.0049597939376961773
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -8.11869121e-02,   2.99755347e+01,  -4.24480949e+01,
        -3.46944695e-10])
  message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 320
      nit: 22
   status: 2
  success: False
        x: array([ 0.00245676,  0.99      ,  1.        ,  1.00000498])
alpha_MLE= 0.00245676156822 beta_MLE 0.989999995 rho_MLE= 0.9999999999 mu_MLE= 1.00000498346
Minimized Criterion: 0.00495979154573


In [10]:
alpha_init = 0.9
beta_init = 0.60
rho_init = 0.2
mu_init = 1
params_init = np.array([alpha_init, beta_init, rho_init, mu_init])
args = (wt,kt,ct,rt)
bounds = ((1e-10,1-1e-10),(1e-10,1-1e-10),(-1+1e-10,1-1e-10),(1e-10,None))
results = opt.minimize(criterion, params_init, args=args, bounds=bounds)
alpha_MLE, beta_MLE, rho_MLE, mu_MLE= results.x
print(results)
print('alpha_MLE=', alpha_MLE, 'beta_MLE', beta_MLE, 'rho_MLE=', rho_MLE, 'mu_MLE=', mu_MLE)
print('Minimized Criterion:', criterion(results.x, *args))

      fun: 0.039234685754041876
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -8.22283116e+00,   8.50245182e+03,  -9.68111272e-02,
        -4.51655789e-01])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 240
      nit: 28
   status: 0
  success: True
        x: array([ 0.93170286,  0.99      ,  0.22897449,  1.00128126])
alpha_MLE= 0.931702857536 beta_MLE 0.989999995038 rho_MLE= 0.228974488618 mu_MLE= 1.00128125527
Minimized Criterion: 0.039234685754


It appears that the estimated parameter values except for $\beta$ are extremely sensitive to initial starting values. The minimized criterion is found when using the answers obtained for problem 2 for $\alpha, \rho, \mu$.